In [393]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [394]:
from sentence_transformers import SentenceTransformer

In [395]:
import os

folder = r"C:\Users\66738\OneDrive - Bain\Documents\Personal Docs\Agents - AI\all-MiniLM-L6-v2"
required = [
    "config.json", "config_sentence_transformers.json", "data_config.json",
    "model.safetensors", "modules.json", "sentence_bert_config.json",
    "special_tokens_map.json", "tokenizer.json", "tokenizer_config.json"
]

print("Missing files:", [f for f in required if not os.path.exists(os.path.join(folder, f))])


Missing files: []


In [396]:
model = SentenceTransformer(
    r"C:\Users\66738\OneDrive - Bain\Documents\Personal Docs\Agents - AI\all-MiniLM-L6-v2"
)
print("Model loaded:", model)

Model loaded: SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [397]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [398]:
# pip install pypdf

In [399]:
import os
os.getcwd()

'c:\\Users\\66738\\OneDrive - Bain\\Documents\\Personal Docs\\Agents - AI\\RAG_\\Personal_projects'

In [400]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: sample.pdf
  ✓ Loaded 2 pages

Total documents loaded: 2


In [401]:
all_pdf_documents

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-06-25T10:57:01+05:30', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_enabled': 'true', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_setdate': '2025-02-19T16:19:54Z', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_method': 'Standard', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_name': 'defa4170-0d19-0005-0004-bc88714345d2', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_siteid': '4bfc8473-b4f8-4a68-83c6-6b4fc5438261', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_actionid': '290907e1-96ba-4391-8e68-48c3808eb77c', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_contentbits': '0', 'author': 'Aditya Gaurav', 'moddate': '2025-06-25T10:57:01+05:30', 'source': '..\\data\\sample.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'sample.pdf', 'file_type': 'pdf'}, page_content='WORK EXPERIENCE \n \nSr. Data Scientist | S

'''chunk_size means “the maximum allowed size of a chunk, measured by whatever your length_function counts.”

If length_function = len → count characters → chunk_size=500 = 500 characters.

If length_function = tokenizer → count tokens → chunk_size=500 = 500 tokens.'''

In [402]:
'''Suppose you have a 4,000-character PDF page.
With chunk_size=1000 and chunk_overlap=200:

Chunk 1: characters 0–1000

Chunk 2: characters 800–1800

Chunk 3: characters 1600–2600

Chunk 4: characters 2400–3400

Chunk 5: characters 3200–4000

✅ Overlap ensures continuity. If a sentence starts at the end of one chunk, it won’t get lost'''

'Suppose you have a 4,000-character PDF page.\nWith chunk_size=1000 and chunk_overlap=200:\n\nChunk 1: characters 0–1000\n\nChunk 2: characters 800–1800\n\nChunk 3: characters 1600–2600\n\nChunk 4: characters 2400–3400\n\nChunk 5: characters 3200–4000\n\n✅ Overlap ensures continuity. If a sentence starts at the end of one chunk, it won’t get lost'

In [403]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=500, chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance.
    
    Splitting priority:
    1. Double newlines (\n\n) → paragraphs
    2. Single newlines (\n) → sentences/line breaks
    3. Spaces (" ") → words
    4. Characters ("") → fallback
    """
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print("\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [404]:
chunks=split_documents(all_pdf_documents)
chunks

Split 2 documents into 33 chunks

Example chunk:
Content: WORK EXPERIENCE 
 
Sr. Data Scientist | SymphonyAI | Dec’ 23-Ongoing 
 
➢  AI for Work : Developed Agentic AI Studio, a low -code/no-code, easily 
customizable SaaS product aimed at improving employee...
Metadata: {'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-06-25T10:57:01+05:30', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_enabled': 'true', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_setdate': '2025-02-19T16:19:54Z', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_method': 'Standard', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_name': 'defa4170-0d19-0005-0004-bc88714345d2', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_siteid': '4bfc8473-b4f8-4a68-83c6-6b4fc5438261', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_actionid': '290907e1-96ba-4391-8e68-48c3808eb77c', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_contentbits': '0', 'au

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-06-25T10:57:01+05:30', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_enabled': 'true', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_setdate': '2025-02-19T16:19:54Z', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_method': 'Standard', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_name': 'defa4170-0d19-0005-0004-bc88714345d2', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_siteid': '4bfc8473-b4f8-4a68-83c6-6b4fc5438261', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_actionid': '290907e1-96ba-4391-8e68-48c3808eb77c', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_contentbits': '0', 'author': 'Aditya Gaurav', 'moddate': '2025-06-25T10:57:01+05:30', 'source': '..\\data\\sample.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'sample.pdf', 'file_type': 'pdf'}, page_content='WORK EXPERIENCE \n \nSr. Data Scientist | S

In [405]:
# pip install -U sentence-transformers
# pip install chromadb


In [406]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [407]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [408]:
class EmbeddingManager():
    def __init__(self, model):
        self.model = model
        self._load_model()

    def _load_model(self):
        try:
           if self.model is None:
            raise ValueError("Model is empty")
        except Exception as e:
            print(f'The model failed to load')

    def create_embedings(self, texts: List[str]) -> np.array:
        if not self.model:
            raise ValueError("Model is empty")
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    
embedding_manager = EmbeddingManager(model)

In [409]:
import os
os.getcwd()

'c:\\Users\\66738\\OneDrive - Bain\\Documents\\Personal Docs\\Agents - AI\\RAG_\\Personal_projects'

In [410]:
class Vector_Store():
    'To store the vectors store'

    def __init__(self, collection_name : str='PDF_documents', persist_directory: str = '../data/vector_store'):
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection=None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            # self.collection = self.client.get_or_create_collection(
            #     name=self.collection_name,
            #     metadata={"description": "PDF document embeddings for RAG"}
            
            try:
                self.client.delete_collection(self.collection_name)
                print(f"Deleted old collection: {self.collection_name}")
            except Exception:
                pass  # OK if it didn't exist

            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
            metadata={
            "description": "PDF document embeddings for RAG",
            "hnsw:space": "cosine"}
)
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            print("Collection metadata:", self.collection.metadata)
            # Expect: {'description': '...', 'hnsw:space': 'cosine'}

            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise


vectorstore = Vector_Store()

Deleted old collection: PDF_documents
Vector store initialized. Collection: PDF_documents
Existing documents in collection: 0
Collection metadata: {'description': 'PDF document embeddings for RAG', 'hnsw:space': 'cosine'}


In [411]:
chunks

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-06-25T10:57:01+05:30', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_enabled': 'true', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_setdate': '2025-02-19T16:19:54Z', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_method': 'Standard', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_name': 'defa4170-0d19-0005-0004-bc88714345d2', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_siteid': '4bfc8473-b4f8-4a68-83c6-6b4fc5438261', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_actionid': '290907e1-96ba-4391-8e68-48c3808eb77c', 'msip_label_defa4170-0d19-0005-0004-bc88714345d2_contentbits': '0', 'author': 'Aditya Gaurav', 'moddate': '2025-06-25T10:57:01+05:30', 'source': '..\\data\\sample.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'sample.pdf', 'file_type': 'pdf'}, page_content='WORK EXPERIENCE \n \nSr. Data Scientist | S

In [412]:
chunks[3].page_content

'•  Used advance d techniques like re -ranking, multi -query, parent document \nretriever, follow-up questions and fine-tuned embedding model (used MRL). \n \n➢  Copilot:  Developed a copilot to automate 70% of ITSM tasks for analysts \nand end users, from creating service requests to resolving tickets. It also \nhandles scheduling meetings, booking flights, applying for leave,  checking \ncompany policies and many more , making it an all -in-one platform for various'

In [413]:
texts=[doc.page_content for doc in chunks]

In [414]:
embeddings = embedding_manager.create_embedings(texts)

Generating embeddings for 33 texts...


Batches: 100%|██████████| 2/2 [00:03<00:00,  1.69s/it]

Generated embeddings with shape: (33, 384)


In [415]:
len(embeddings)

33

In [416]:
# type(embeddings)
embeddings[11]

array([-8.29159841e-02,  4.37366590e-02, -5.09156734e-02, -3.67763638e-02,
       -6.61081895e-02,  3.83738168e-02, -1.02012511e-03,  5.67821562e-02,
        8.43034759e-02,  7.67126158e-02,  5.83771877e-02,  3.19137722e-02,
        3.47713451e-03,  1.40915215e-02,  8.27813987e-03, -3.95155251e-02,
        5.17761000e-02,  2.17157919e-02,  8.78092833e-03,  2.78533455e-02,
       -2.23106481e-02, -1.66787039e-02, -4.09613252e-02, -3.89671475e-02,
        5.34703732e-02, -3.56080681e-02, -6.13550954e-02,  3.15687098e-02,
        2.22488623e-02, -3.96593325e-02, -5.94637096e-02,  7.44817732e-03,
        7.03805983e-02,  4.44233492e-02,  5.50222099e-02, -3.71679477e-02,
       -5.34124933e-02, -1.00175459e-02, -2.19375975e-02,  3.14181969e-02,
        6.23581037e-02, -4.32755686e-02, -1.04920432e-01, -3.57913673e-02,
       -1.34633924e-03, -3.53986509e-02, -7.49409944e-02,  1.16947986e-01,
       -2.84806751e-02,  4.69128639e-02, -6.49068952e-02,  2.52130702e-02,
       -2.26875059e-02,  

In [417]:
vectorstore.add_documents(chunks,embeddings)

Adding 33 documents to vector store...
Successfully added 33 documents to vector store
Total documents in collection: 33


In [430]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vectorstore: Vector_Store, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vectorstore: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vectorstore = vectorstore
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.create_embedings([query])[0]
        # print(query_embedding)
        
        # Search in vector store
        try:
            results = self.vectorstore.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return [(doc['content'], doc['similarity_score']) for doc in retrieved_docs]
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

In [431]:
rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [432]:
rag_retriever.retrieve("in how many compnaies you have worked in")

Retrieving documents for query: 'in how many compnaies you have worked in'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 36.82it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[('•  Used advance d techniques like re -ranking, multi -query, parent document \nretriever, follow-up questions and fine-tuned embedding model (used MRL). \n \n➢  Copilot:  Developed a copilot to automate 70% of ITSM tasks for analysts \nand end users, from creating service requests to resolving tickets. It also \nhandles scheduling meetings, booking flights, applying for leave,  checking \ncompany policies and many more , making it an all -in-one platform for various',
  0.7297980785369873),
 ('WORK EXPERIENCE \n \nSr. Data Scientist | SymphonyAI | Dec’ 23-Ongoing \n \n➢  AI for Work : Developed Agentic AI Studio, a low -code/no-code, easily \ncustomizable SaaS product aimed at improving employee productivity . The \nplatform allows users to define a flow by dragging components onto a canvas, \nseamlessly connecting them to agents to trigger their agentic flow.  \n \n•  Built custom components /MCP servers  for HR, IT, coding assistance and',
  0.7402739524841309),
 ('below the MMP m